In [1]:
import os
os.chdir('/mnt/ufs18/home-230/pandavis/cellular_resilience/CellResilienceModel/codebase')


import re
import random
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna

from src.linear_regression import ElasticLinear
from src.train import train
from src.mlp import MLP
from src.evaluate import evaluate

import torch
from torch.utils.data import TensorDataset, DataLoader
import lightning as L
import pickle

seed = 42
random.seed(seed)
np.random.seed(seed)

/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/mnt/ufs18/home-230/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
banksy_matrix = sc.read_h5ad('../data/BreastCancer10xGenomics_Rep1/exported_data/banksy_matrix.h5ad')
banksy_matrix

AnnData object with n_obs × n_vars = 156831 × 900
    obs: 'celltype', 'x_centroid', 'y_centroid', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_10_genes', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'is_nbr', 'k'

In [3]:
pd.set_option('display.max_rows', None)
print(banksy_matrix.to_df().columns[0:300])
print(banksy_matrix.to_df().columns[300:600])
print(banksy_matrix.to_df().columns[600:])

Index(['ABCC11', 'ACTA2', 'ACTG2', 'ADAM9', 'ADGRE5', 'ADH1B', 'ADIPOQ',
       'AGR3', 'AHSP', 'AIF1',
       ...
       'TUBB2B', 'TYROBP', 'UCP1', 'USP53', 'VOPP1', 'VWF', 'WARS', 'ZEB1',
       'ZEB2', 'ZNF562'],
      dtype='object', length=300)
Index(['ABCC11_nbr_0', 'ACTA2_nbr_0', 'ACTG2_nbr_0', 'ADAM9_nbr_0',
       'ADGRE5_nbr_0', 'ADH1B_nbr_0', 'ADIPOQ_nbr_0', 'AGR3_nbr_0',
       'AHSP_nbr_0', 'AIF1_nbr_0',
       ...
       'TUBB2B_nbr_0', 'TYROBP_nbr_0', 'UCP1_nbr_0', 'USP53_nbr_0',
       'VOPP1_nbr_0', 'VWF_nbr_0', 'WARS_nbr_0', 'ZEB1_nbr_0', 'ZEB2_nbr_0',
       'ZNF562_nbr_0'],
      dtype='object', length=300)
Index(['ABCC11_nbr_1', 'ACTA2_nbr_1', 'ACTG2_nbr_1', 'ADAM9_nbr_1',
       'ADGRE5_nbr_1', 'ADH1B_nbr_1', 'ADIPOQ_nbr_1', 'AGR3_nbr_1',
       'AHSP_nbr_1', 'AIF1_nbr_1',
       ...
       'TUBB2B_nbr_1', 'TYROBP_nbr_1', 'UCP1_nbr_1', 'USP53_nbr_1',
       'VOPP1_nbr_1', 'VWF_nbr_1', 'WARS_nbr_1', 'ZEB1_nbr_1', 'ZEB2_nbr_1',
       'ZNF562_nbr_1'],
      dtype='

In [4]:
banksy_matrix.obs['celltype'].value_counts()

celltype
Luminal epithelial cells      61326
T cells                       28479
Fibroblasts                   20672
Macrophages                   14007
Endothelial cells             12276
B cells                        9709
Myoepithelial cells            9394
Cancer cells                    968
Name: count, dtype: int64

In [5]:
celltype_adata = banksy_matrix[banksy_matrix.obs['celltype'] == 'Luminal epithelial cells  ']
celltype_adata

View of AnnData object with n_obs × n_vars = 61326 × 900
    obs: 'celltype', 'x_centroid', 'y_centroid', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_10_genes', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'is_nbr', 'k'

In [6]:
n = len(banksy_matrix.var) // 3

X = celltype_adata.X[:, n:2*n]
G = celltype_adata.X[:, :n]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, G, test_size=0.2, random_state=42
)

In [8]:
'''
results = train(
    MLP,
    X_train,
    y_train,
    X_test,
    y_test,
    loss_fn=torch.nn.MSELoss(),
    learning_rate=0.05,
    max_epochs=100,
    batch_size=None,
    #early_stopping=True,  
   # patience=10,
)
'''

'\nresults = train(\n    MLP,\n    X_train,\n    y_train,\n    X_test,\n    y_test,\n    loss_fn=torch.nn.MSELoss(),\n    learning_rate=0.05,\n    max_epochs=100,\n    batch_size=None,\n    #early_stopping=True,  \n   # patience=10,\n)\n'

In [ ]:
#evaluate(results['model'], X_test, y_test)

In [17]:
cell_types = [
    "Luminal epithelial cells  ",
    "T cells  ",
    "Fibroblasts  ",
    "Macrophages  ",
    "Endothelial cells  ",
    "B cells  ",
    "Myoepithelial cells  ",
]

weights = {}

for cell_type in cell_types:
    print(f"Processing {cell_type}...")

    celltype_adata = banksy_matrix[banksy_matrix.obs['celltype'] == cell_type]

    #n = len(banksy_matrix.var) // 3
    X = celltype_adata.X[:, n:2*n]
    G = celltype_adata.X[:, :n]

    X_train, X_test, y_train, y_test = train_test_split(X, G, test_size=0.2, random_state=42)

    # Train MLP model
    results = train(
                    MLP,
                    X_train,
                    y_train,
                    X_test,
                    y_test,
                    loss_fn=torch.nn.MSELoss(),
                    learning_rate=0.05,
                    max_epochs=100,
                    batch_size=None,
                    #early_stopping=True,  
                   # patience=10,
                )

    metrics = evaluate(
        model=results['model'], 
        X_test=X_test, 
        y_test=y_test
    )
    
   
    model = results['model']
    layer_weights = []
    
    for layer in model.model:
        weight = layer.weight.detach().cpu().numpy()
        bias = layer.bias.detach().cpu().numpy()
        layer_weights.append({"weight": weight, "bias": bias})

    
    weights[cell_type] = layer_weights # This one stores None for now.
    
    #np.save(f"results/{cell_type}_mlp_weights.npy", results["weights"])
    print(f"MLP Metrics for {cell_type}: {metrics}")

Processing Luminal epithelial cells  ...


INFO: Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO:lightning.pytorch.utilities.rank_zero:Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/home/pandavis/anaconda3/envs/pytorch/lib/python ...
INFO: GPU available: True (cuda)

Training: |                                                                                                   …

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


MLP Metrics for Luminal epithelial cells  : {'RMSE': 0.5311313, 'R2': -0.03000536755965161}
Processing T cells  ...


...

MLP Metrics for B cells  : {'RMSE': 1.085326, 'R2': 0.008468816532699262}
Processing Myoepithelial cells  ...


/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
INFO: 
  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | loss_fn | MSELoss    | 0      | train
1 | model   | Sequential | 66.3 K | train
-----------------------------------------------
66.3 K    Trainable params
0         Non-trainable params
66.3 K    Total params
0.265     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | loss_fn | MSELoss    | 0      | train
1 | model   | Sequential | 66.3 K | train
--

Training: |                                                                                                   …

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


MLP Metrics for Myoepithelial cells  : {'RMSE': 0.86678183, 'R2': 0.03189465882546596}


In [19]:
model = results['model']

In [20]:
for layer in model.model:
        weight = layer.weight.detach().cpu().numpy()
        bias = layer.bias.detach().cpu().numpy()
        layer_weights.append({"weight": weight, "bias": bias})

In [34]:
fro_norms = {}

for cell_type in cell_types:
    weight_celltype = weights[cell_type] 
    fro_norms[cell_type] = {"layer_norms": [], "total_norm": 0}  
    total_fro_norm = 0  
    for layer_index, layer in enumerate(weight_celltype):
        W = layer["weight"]  
        fro_norm = np.linalg.norm(W, ord="fro")  
        fro_norms[cell_type]["layer_norms"].append(fro_norm)  
        total_fro_norm += fro_norm  
    
    fro_norms[cell_type]["total_norm"] = total_fro_norm  # Store total norm

for cell_type, data in fro_norms.items():
    #print(f"Frobenius norms for {cell_type}: {data['layer_norms']}")
    print(f"Frobenius norm for {cell_type}: {data['total_norm']}")

Total Frobenius norm for Luminal epithelial cells  : 79.04887390136719
Total Frobenius norm for T cells  : 91.46709251403809
Total Frobenius norm for Fibroblasts  : 87.55792045593262
Total Frobenius norm for Macrophages  : 98.32486152648926
Total Frobenius norm for Endothelial cells  : 90.76177597045898
Total Frobenius norm for B cells  : 87.98000907897949
Total Frobenius norm for Myoepithelial cells  : 171.36374282836914
